# Heat Transmission Calculation Notebook

This notebook sets up a Python environment and demonstrates interactive forms and dropdowns for heat transmission calculations using Jupyter widgets.

## Install and Import Jupyter Widgets

The following cell ensures `ipywidgets` is installed and imports the necessary modules for interactive widgets.

In [1]:
# Install ipywidgets if not already installed (uncomment if needed)
# !pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display

## Warmtedoorgangscoëfficiënt (U-waarde)

$$U = \frac{1}{R_e + R_c + R_i} \quad \left[\frac{W}{m^2 \cdot K}\right]$$

| Symbool | Omschrijving | Standaardwaarde |
|---------|-------------|-----------------|
| $R_i$ | Warmte-overgangsweerstand binnenzijde | 0,13 m²·K/W |
| $R_e$ | Warmte-overgangsweerstand buitenzijde | 0,04 m²·K/W |
| $R_c$ | Warmteweerstand constructie | $\sum d / \lambda$ |

Voeg lagen toe via de knop hieronder. Per laag kies je een materiaal uit de lijst of vul je de R-waarde handmatig in (voor bijv. luchtspouw).


In [2]:
import json
import importlib
import ipywidgets as widgets
from IPython.display import display, HTML

import heat_calc
importlib.reload(heat_calc)                          # pick up any edits without restarting kernel
from heat_calc import SURFACE_R, LayerWidget

# ── Load material data ────────────────────────────────────────────────────────
with open('material_properties.json', 'r', encoding='utf-8') as f:
    materials = json.load(f)

# ── UI state ──────────────────────────────────────────────────────────────────
layers     = []
layers_box = widgets.VBox([])
out        = widgets.Output()

ri_dd = widgets.Dropdown(
    options=list(SURFACE_R.keys()),
    value='Binnenzijde  —  Ri = 0,13 m²·K/W',
    description='Ri (binnen):',
    layout=widgets.Layout(width='340px')
)
re_dd = widgets.Dropdown(
    options=list(SURFACE_R.keys()),
    value='Buitenzijde  —  Re = 0,04 m²·K/W',
    description='Re (buiten):',
    layout=widgets.Layout(width='340px')
)
ri_dd.observe(lambda _: refresh(), names='value')
re_dd.observe(lambda _: refresh(), names='value')

add_btn = widgets.Button(
    description='＋ Voeg laag toe', button_style='primary',
    layout=widgets.Layout(width='160px')
)


def refresh(*_):
    """Recompute totals and render the result table."""
    ri = SURFACE_R[ri_dd.value]
    re = SURFACE_R[re_dd.value]

    with out:
        out.clear_output(wait=True)

        rows_html = ''
        total_d   = 0.0
        total_rc  = 0.0

        rows_html += (
            f'<tr class="surface"><td>lucht (binnen)</td>'
            f'<td>—</td><td>—</td><td>—</td><td>{ri:.2f}</td></tr>\n'
        )

        for layer in layers:
            info  = layer.row_info()
            r     = info['R']
            d     = info['d']
            d_str = f'{d:.3f}' if isinstance(d, float) else '—'
            r_str = f'{r:.3f}' if r is not None else '?'
            if isinstance(d, float): total_d  += d
            if r is not None:        total_rc += r
            rows_html += (
                f'<tr><td>{info["naam"]}</td>'
                f'<td>{d_str}</td><td>{info["lam"]}</td>'
                f'<td>{r_str}</td><td>—</td></tr>\n'
            )

        rows_html += (
            f'<tr class="surface"><td>lucht (buiten)</td>'
            f'<td>—</td><td>—</td><td>—</td><td>{re:.2f}</td></tr>\n'
        )

        total_r = ri + total_rc + re
        u       = 1.0 / total_r if total_r > 0 else None
        u_str   = f'{u:.3f}' if u is not None else '?'
        d_tot   = f'{total_d:.3f}' if total_d > 0 else '—'

        html = f"""
<style>
  .utbl {{ border-collapse:collapse; font-family:sans-serif;
           font-size:13px; min-width:680px; }}
  .utbl th {{ background:#2c5f8a; color:white;
              padding:6px 12px; text-align:left; }}
  .utbl td {{ padding:5px 12px; border-bottom:1px solid #ddd; }}
  .utbl tr:not(.total-row):not(.u-row):hover td {{ background:#f0f7ff; }}
  .utbl .surface td  {{ color:#555; font-style:italic; }}
  .utbl .total-row td {{ font-weight:bold; background:#e8f0fe;
                         border-top:2px solid #2c5f8a; }}
  .utbl .u-row td   {{ font-weight:bold; background:#2c5f8a;
                       color:white; font-size:15px; }}
</style>
<table class="utbl">
  <tr>
    <th>Materiaal / Laag</th>
    <th>d [m]</th>
    <th>λ [W/(m·K)]</th>
    <th>R = d/λ [m²·K/W]</th>
    <th>Ri &amp; Re [m²·K/W]</th>
  </tr>
  {rows_html}
  <tr class="total-row">
    <td>TOTAAL</td>
    <td>{d_tot}</td>
    <td>—</td>
    <td>{total_rc:.3f}</td>
    <td>{ri + re:.2f}</td>
  </tr>
  <tr class="u-row">
    <td colspan="5">
      U = 1 / (R<sub>i</sub> {ri:.2f} + R<sub>c</sub> {total_rc:.3f}
              + R<sub>e</sub> {re:.2f})
      &nbsp;=&nbsp; <b>{u_str} W/(m²·K)</b>
    </td>
  </tr>
</table>"""
        display(HTML(html))


def add_layer(_=None):
    layer = LayerWidget(materials, update_cb=refresh, remove_cb=remove_layer)
    layers.append(layer)
    layers_box.children = [l.box for l in layers]
    refresh()


def remove_layer(layer):
    layers.remove(layer)
    layers_box.children = [l.box for l in layers]
    refresh()


add_btn.on_click(add_layer)

display(widgets.VBox([
    widgets.HTML('<b>Overgangsweerstanden</b>'),
    widgets.HBox([ri_dd, re_dd]),
    widgets.HTML('<b style="margin-top:10px;display:block">Constructielagen (Rc)</b>'),
    layers_box,
    add_btn,
    widgets.HTML('<b style="margin-top:10px;display:block">Resultaat</b>'),
    out,
]))

# Start with one empty layer
add_layer()


## Correctiefactoren f_k en f_ig,k

Bereken de correctiefactoren voor warmteverlies:

- **f_k** – Warmteverlies via een onverwarmde ruimte: $H_{T,iue} = A \cdot U \cdot f_k$ [W/K]
- **f_ig,k** – Warmteverlies via de grond: $H_{T,ig} = A \cdot U_{equiv,k} \cdot f_{ig,k} \cdot f_{gw}$ [W/K]

Kies hieronder het berekeningstype en volg de invoervelden.


In [ ]:
import importlib
import ipywidgets as widgets
from IPython.display import display, HTML

import heat_calc
importlib.reload(heat_calc)
from heat_calc import (
    HEATING_SYSTEMS, ROOM_TEMPERATURES_WOON, ROOM_TEMPERATURES_SENIOREN,
    THETA_E_DEFAULT, THETA_ME_DEFAULT,
    get_delta_theta, calculate_fk_formula,
    lookup_fk_table_2_3, lookup_fk_table_2_13,
    get_f_gw, calculate_fig_k, get_u_equiv_k,
    calculate_h_t_ig, validate_fk,
)

# ── Shared output area ────────────────────────────────────────────────────────
fk_out = widgets.Output()

# ── Step 0: Calculation-type toggle ───────────────────────────────────────────
calc_type = widgets.RadioButtons(
    options=['f_k (onverwarmde ruimte)', 'f_ig,k (grond)'],
    description='Berekeningstype:',
    layout=widgets.Layout(width='auto'),
)

# ── Shared inputs (§14.1) ────────────────────────────────────────────────────
room_type_opts = (
    ['— Kies ruimtetype —']
    + [f'Woon: {k}' for k in ROOM_TEMPERATURES_WOON]
    + [f'Senioren: {k}' for k in ROOM_TEMPERATURES_SENIOREN]
)
room_type_dd = widgets.Dropdown(
    options=room_type_opts, description='Ruimtetype:',
    layout=widgets.Layout(width='460px'),
)
theta_i_input = widgets.FloatText(value=22.0, description='θ_i [°C]:',
                                  layout=widgets.Layout(width='200px'))
theta_e_input = widgets.FloatText(value=THETA_E_DEFAULT, description='θ_e [°C]:',
                                  layout=widgets.Layout(width='200px'))

def _on_room_type(change):
    val = change['new']
    if val.startswith('Woon: '):
        key = val[len('Woon: '):]
        theta_i_input.value = float(ROOM_TEMPERATURES_WOON[key])
    elif val.startswith('Senioren: '):
        key = val[len('Senioren: '):]
        theta_i_input.value = float(ROOM_TEMPERATURES_SENIOREN[key])

room_type_dd.observe(_on_room_type, names='value')

shared_box = widgets.VBox([
    widgets.HTML('<b>Gemeenschappelijke invoer</b>'),
    room_type_dd,
    widgets.HBox([theta_i_input, theta_e_input]),
])

# ══════════════════════════════════════════════════════════════════════════════
# DEEL 1 – f_k  panels
# ══════════════════════════════════════════════════════════════════════════════

# ── f_k sub-choice: Is θ_a known? ────────────────────────────────────────────
theta_a_known = widgets.RadioButtons(
    options=['Ja (Scenario A – formule)', 'Nee (Scenario B of C – tabel)'],
    description='Is θ_a bekend?',
    layout=widgets.Layout(width='auto'),
)

# ── Scenario A inputs ─────────────────────────────────────────────────────────
theta_a_input = widgets.FloatText(value=5.0, description='θ_a [°C]:',
                                  layout=widgets.Layout(width='200px'))
component_a_dd = widgets.Dropdown(
    options=['Wand', 'Vloer', 'Plafond'], description='Bouwdeel:',
    layout=widgets.Layout(width='250px'),
)
heating_a_dd = widgets.Dropdown(
    options=list(HEATING_SYSTEMS.keys()), description='Verwarming:',
    layout=widgets.Layout(width='500px'),
)
calc_a_btn = widgets.Button(description='Bereken f_k (A)',
                            button_style='success',
                            layout=widgets.Layout(width='180px'))

scenario_a_box = widgets.VBox([
    widgets.HTML('<b>Scenario A – f_k via formule (θ_a bekend)</b>'),
    theta_a_input,
    widgets.HBox([component_a_dd, heating_a_dd]),
    calc_a_btn,
])

def _toggle_heating_a(change):
    heating_a_dd.layout.visibility = (
        'hidden' if change['new'] == 'Wand' else 'visible')

component_a_dd.observe(_toggle_heating_a, names='value')
_toggle_heating_a({'new': component_a_dd.value})

def _calc_a(_):
    with fk_out:
        fk_out.clear_output(wait=True)
        comp = component_a_dd.value.lower()
        hs = heating_a_dd.value if comp in ('vloer', 'plafond') else None
        try:
            fk = calculate_fk_formula(
                theta_i_input.value, theta_e_input.value,
                theta_a_input.value, comp, heating_system=hs)
            warn = validate_fk(fk)
            dth = get_delta_theta(hs, comp) if comp in ('vloer', 'plafond') else 0.0
            html = (f"<h4>Resultaat Scenario A</h4>"
                    f"<b>Bouwdeel:</b> {component_a_dd.value}<br>")
            if comp != 'wand':
                label = 'Δθ_1' if comp == 'plafond' else 'Δθ_2'
                html += f"<b>{label}:</b> {dth:+.1f} K<br>"
            html += f"<b>f_k = {fk:.4f}</b><br>"
            if warn:
                html += f'<span style="color:orange">⚠ {warn}</span>'
            display(HTML(html))
        except Exception as e:
            display(HTML(f'<span style="color:red">Fout: {e}</span>'))

calc_a_btn.on_click(_calc_a)

# ── Scenario B / C sub-choice ─────────────────────────────────────────────────
bc_purpose = widgets.RadioButtons(
    options=['Warmteverlies (Scenario B – Tabel 2.3)',
             'Tijdconstante (Scenario C – Tabel 2.13)'],
    description='Doel:',
    layout=widgets.Layout(width='auto'),
)

# ── Scenario B inputs ─────────────────────────────────────────────────────────
b_category = widgets.Dropdown(
    options=['Vertrek / ruimte', 'Ruimte onder het dak',
             'Gemeenschappelijke verkeersruimte', 'Vloer boven kruipruimte'],
    description='Type ruimte:',
    layout=widgets.Layout(width='400px'),
)
# Sub-inputs per category
b_ext_walls = widgets.Dropdown(options=['1', '2', '3+'],
                               description='Gevels:', value='1',
                               layout=widgets.Layout(width='200px'))
b_ext_door = widgets.Dropdown(options=['Nee', 'Ja'],
                              description='Buitendeur?',
                              layout=widgets.Layout(width='200px'))
b_roof_type = widgets.Dropdown(
    options=['Pannendak zonder folie', 'Niet-geïsoleerd', 'Geïsoleerd'],
    description='Daktype:',
    layout=widgets.Layout(width='300px'),
)
b_has_ext_walls = widgets.Dropdown(options=['Ja', 'Nee'],
                                   description='Buitenwanden?',
                                   layout=widgets.Layout(width='200px'))
b_vent_rate = widgets.FloatText(value=0.3, description='Ventilatievoud:',
                                layout=widgets.Layout(width='200px'))
b_a_opening_v = widgets.FloatText(value=0.003, description='A_opening/V:',
                                  layout=widgets.Layout(width='200px'))
b_opening_size = widgets.Dropdown(
    options=['Zwak (≤ 1.000 mm²/m²)',
             'Matig (> 1.000, ≤ 1.500 mm²/m²)',
             'Sterk (> 1.500 mm²/m²)'],
    description='Ventilatie:',
    layout=widgets.Layout(width='380px'),
)

b_sub_vertrek = widgets.VBox([widgets.HBox([b_ext_walls, b_ext_door])])
b_sub_dak = widgets.VBox([b_roof_type])
b_sub_verkeer = widgets.VBox([widgets.HBox([b_has_ext_walls, b_vent_rate,
                                             b_a_opening_v])])
b_sub_kruip = widgets.VBox([b_opening_size])

b_sub_container = widgets.VBox([b_sub_vertrek])

calc_b_btn = widgets.Button(description='Bereken f_k (B)',
                            button_style='success',
                            layout=widgets.Layout(width='180px'))

scenario_b_box = widgets.VBox([
    widgets.HTML('<b>Scenario B – f_k via Tabel 2.3 (warmteverlies)</b>'),
    b_category, b_sub_container, calc_b_btn,
])

def _toggle_b_sub(change):
    mapping = {
        'Vertrek / ruimte': b_sub_vertrek,
        'Ruimte onder het dak': b_sub_dak,
        'Gemeenschappelijke verkeersruimte': b_sub_verkeer,
        'Vloer boven kruipruimte': b_sub_kruip,
    }
    b_sub_container.children = [mapping.get(change['new'], widgets.VBox())]

b_category.observe(_toggle_b_sub, names='value')

def _on_b_walls(change):
    b_ext_door.layout.visibility = 'visible' if change['new'] == '2' else 'hidden'
b_ext_walls.observe(_on_b_walls, names='value')
_on_b_walls({'new': b_ext_walls.value})

def _calc_b(_):
    with fk_out:
        fk_out.clear_output(wait=True)
        cat_label = b_category.value
        try:
            if cat_label == 'Vertrek / ruimte':
                walls = 3 if b_ext_walls.value == '3+' else int(b_ext_walls.value)
                fk = lookup_fk_table_2_3(
                    'vertrek', external_walls=walls,
                    exterior_door=(b_ext_door.value == 'Ja'))
            elif cat_label == 'Ruimte onder het dak':
                roof_map = {
                    'Pannendak zonder folie': 'pannendak_zonder_folie',
                    'Niet-geïsoleerd': 'niet_geisoleerd',
                    'Geïsoleerd': 'geisoleerd',
                }
                fk = lookup_fk_table_2_3(
                    'dak', roof_type=roof_map[b_roof_type.value])
            elif cat_label == 'Gemeenschappelijke verkeersruimte':
                fk = lookup_fk_table_2_3(
                    'verkeersruimte',
                    has_exterior_walls=(b_has_ext_walls.value == 'Ja'),
                    ventilation_rate=b_vent_rate.value,
                    a_opening_v=b_a_opening_v.value)
            else:  # Kruipruimte
                size_map = {
                    'Zwak (≤ 1.000 mm²/m²)': 'zwak',
                    'Matig (> 1.000, ≤ 1.500 mm²/m²)': 'matig',
                    'Sterk (> 1.500 mm²/m²)': 'sterk',
                }
                fk = lookup_fk_table_2_3(
                    'kruipruimte',
                    opening_size=size_map[b_opening_size.value])
            display(HTML(
                f"<h4>Resultaat Scenario B (Tabel 2.3)</h4>"
                f"<b>Categorie:</b> {cat_label}<br>"
                f"<b>f_k = {fk:.1f}</b>"))
        except Exception as e:
            display(HTML(f'<span style="color:red">Fout: {e}</span>'))

calc_b_btn.on_click(_calc_b)

# ── Scenario C inputs ─────────────────────────────────────────────────────────
c_space_type = widgets.Dropdown(
    options=['Kelder', 'Stallingsruimte',
             'Kruipruimte, serre, trappenhuis'],
    description='Ruimtetype:',
    layout=widgets.Layout(width='400px'),
)
calc_c_btn = widgets.Button(description='Bereken f_k (C)',
                            button_style='success',
                            layout=widgets.Layout(width='180px'))

scenario_c_box = widgets.VBox([
    widgets.HTML('<b>Scenario C – f_k via Tabel 2.13 (tijdconstante)</b>'),
    c_space_type, calc_c_btn,
])

def _calc_c(_):
    with fk_out:
        fk_out.clear_output(wait=True)
        label = c_space_type.value
        # Map composite option to individual key
        key_map = {
            'Kelder': 'kelder',
            'Stallingsruimte': 'stallingsruimte',
            'Kruipruimte, serre, trappenhuis': 'kruipruimte',
        }
        try:
            fk = lookup_fk_table_2_13(key_map[label])
            display(HTML(
                f"<h4>Resultaat Scenario C (Tabel 2.13)</h4>"
                f"<b>Type:</b> {label}<br>"
                f"<b>f_k = {fk:.1f}</b>"))
        except Exception as e:
            display(HTML(f'<span style="color:red">Fout: {e}</span>'))

calc_c_btn.on_click(_calc_c)

# ── Combine B/C with purpose toggle ──────────────────────────────────────────
bc_container = widgets.VBox([scenario_b_box])

def _toggle_bc(change):
    if 'Warmteverlies' in change['new']:
        bc_container.children = [scenario_b_box]
    else:
        bc_container.children = [scenario_c_box]

bc_purpose.observe(_toggle_bc, names='value')

scenario_bc_box = widgets.VBox([bc_purpose, bc_container])

# ── f_k panel: combine A vs B/C ──────────────────────────────────────────────
fk_panel_container = widgets.VBox([scenario_a_box])

def _toggle_fk_panel(change):
    if 'Ja' in change['new']:
        fk_panel_container.children = [scenario_a_box]
    else:
        fk_panel_container.children = [scenario_bc_box]

theta_a_known.observe(_toggle_fk_panel, names='value')

fk_panel = widgets.VBox([theta_a_known, fk_panel_container])

# ══════════════════════════════════════════════════════════════════════════════
# DEEL 2 – f_ig,k  panel
# ══════════════════════════════════════════════════════════════════════════════

theta_me_input = widgets.FloatText(
    value=THETA_ME_DEFAULT, description='θ_me [°C]:',
    layout=widgets.Layout(width='200px'))
heated_on_ground = widgets.Dropdown(
    options=['Nee', 'Ja'],
    description='Verwarmd op grond?',
    layout=widgets.Layout(width='250px'),
)
component_e_dd = widgets.Dropdown(
    options=['Wand', 'Vloer'], description='Bouwdeel:',
    layout=widgets.Layout(width='200px'),
)
heating_e_dd = widgets.Dropdown(
    options=list(HEATING_SYSTEMS.keys()), description='Verwarming:',
    layout=widgets.Layout(width='500px'),
)
gw_depth_dd = widgets.Dropdown(
    options=['≥ 1 m onder vloer (f_gw = 1,00)',
             '< 1 m onder vloer (f_gw = 1,15)'],
    description='Grondwater:',
    layout=widgets.Layout(width='420px'),
)
rc_input = widgets.FloatText(value=3.5, description='R_c [m²·K/W]:',
                             layout=widgets.Layout(width='200px'))
area_input = widgets.FloatText(value=0.0, description='A [m²]:',
                               layout=widgets.Layout(width='200px'),
                               step=0.1)

calc_e_btn = widgets.Button(description='Bereken f_ig,k',
                            button_style='success',
                            layout=widgets.Layout(width='180px'))

def _toggle_heating_e(change):
    heating_e_dd.layout.visibility = (
        'hidden' if change['new'] == 'Wand' else 'visible')

component_e_dd.observe(_toggle_heating_e, names='value')
_toggle_heating_e({'new': component_e_dd.value})

def _calc_e(_):
    with fk_out:
        fk_out.clear_output(wait=True)
        heated = (heated_on_ground.value == 'Ja')
        comp = component_e_dd.value.lower()
        hs = heating_e_dd.value if comp == 'vloer' else None
        gw_deep = '≥ 1 m' in gw_depth_dd.value
        try:
            fig = calculate_fig_k(
                theta_i_input.value, theta_e_input.value,
                theta_me_input.value, comp,
                heating_system=hs, heated_element_on_ground=heated)
            f_gw = get_f_gw(1.0 if gw_deep else 0.5)
            u_eq = get_u_equiv_k(rc_input.value)
            html = (f"<h4>Resultaat Scenario E (f_ig,k)</h4>"
                    f"<b>Bouwdeel:</b> {component_e_dd.value}<br>")
            if heated:
                html += "<b>Verwarmde wand/vloer op grond → f_ig,k = 0</b><br>"
            else:
                if comp == 'vloer' and hs:
                    dth2 = get_delta_theta(hs, 'vloer')
                    html += f"<b>Δθ_2:</b> {dth2:+.1f} K<br>"
                html += f"<b>f_ig,k = {fig:.4f}</b><br>"
            html += (f"<b>f_gw = {f_gw:.2f}</b><br>"
                     f"<b>U_equiv,k = {u_eq:.2f} W/(m²·K)</b> "
                     f"(R_c = {rc_input.value:.2f})<br>")
            a = area_input.value
            if a > 0:
                h = calculate_h_t_ig(a, u_eq, fig, f_gw)
                html += (f"<b>H_T,ig = {a:.2f} × {u_eq:.2f} × "
                         f"{fig:.4f} × {f_gw:.2f} = {h:.4f} W/K</b>")
            display(HTML(html))
        except Exception as e:
            display(HTML(f'<span style="color:red">Fout: {e}</span>'))

calc_e_btn.on_click(_calc_e)

figk_panel = widgets.VBox([
    widgets.HTML('<b>Scenario E – f_ig,k via formule</b>'),
    theta_me_input,
    widgets.HBox([component_e_dd, heating_e_dd]),
    heated_on_ground,
    gw_depth_dd,
    widgets.HBox([rc_input, area_input]),
    calc_e_btn,
])

# ══════════════════════════════════════════════════════════════════════════════
# Main panel switch
# ══════════════════════════════════════════════════════════════════════════════
main_panel = widgets.VBox([fk_panel])

def _switch_main(change):
    if 'f_k' in change['new'] and 'f_ig' not in change['new']:
        main_panel.children = [fk_panel]
    else:
        main_panel.children = [figk_panel]

calc_type.observe(_switch_main, names='value')

display(widgets.VBox([
    calc_type,
    shared_box,
    main_panel,
    widgets.HTML('<hr><b>Resultaat</b>'),
    fk_out,
]))